# このcodeの目的
コンペの概要と、データの内容を最低限理解する。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from PIL import Image
import os, shutil
from tqdm import tqdm
import math
import cv2

# 1. コンペ概要
がんの放射線治療では、胃や腸を避けて腫瘍へX線を照射しなければならない。
そのために、放射線腫瘍医は、胃や腸の位置の輪郭を手動で描画している。
この作業には、1日15分から1時間かかり、自動化できればより迅速に効果的な治療ができるようになる。
実際のがん患者のMRI画像（別日に1~5回撮影）を用いて、胃や腸を自動でセグメンテーションするモデルを作ってください。

## 評価方法
- Dice係数<br>
2つの集合の平均要素数と共通要素数の割合で、1に近づくほど高スコア<br>
$$
\frac{2*|X\cap Y|}{|X|+|Y|}\\
$$
$X$:予測したピクセルの集合<br>
$Y$:正解ピクセルの集合<br>

- 3Dハウスドルフ距離<br>
スライス深度をZ座標とする各2次元セグメンテーションを組み合わせて3次元ボリュームを構成し、その間のハウスドルフ距離を求める。<br>
ハウスドルフ距離は、集合$X$のどの点であっても、距離$d$の範囲内に集合$Y$のいずれかの点に到達できる最短距離
$$
\max_{x\in X} \{\min_{y\in Y} \{d(x,y)\}\}
$$
更に、画像サイズで正規化することで、0-1のスコアに変換<br>

### 評価スコア算出
Dice係数：0.4、3Dハウスドルフ距離スコア：0.6 の重みで加重平均してスコアを算出

# 2. train.csv
全てのトレーニングセットに対する、RLEエンコードされたマスク情報<br>
いくつかのケースは時間によってtrainとtestが分割され（初期の日はトレーニング、後の日はテスト）、いくつかのケースはケースによってtrainとtestが分割される（ケースの全体がトレーニングまたはテストに含まれる）。

In [ ]:
df_train = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/train.csv')
df_train[df_train['segmentation'].notna()].head()

id : 画像のid、（case 患者の番号、day 撮影日、slice 断面番号）<br>
class : 分類<br>
segmentation : セグメンテーションピクセル情報

In [ ]:
df_train.describe()

In [ ]:
# 念のため欠損値ないこと確認
df_train.isnull().sum()

In [ ]:
# 分類ラベルの内容確認
df_train['class'].value_counts()

idの数と同数存在<br>
large_bowel: 大腸<br>
small_bowel: 小腸<br>
stomach: 胃

# 3. train画像データ
16bitグレースケールPNG形式<br>
特定のケース、日ごとにフォルダが分かれており、フォルダ内にスキャンスライスの複数の画像が格納されている。<br>
ファイル名の4つの数値は、スライスの高さ/幅（ピクセル単位の整数）と、高さ/幅のピクセル間隔（mm単位の浮動小数点）を表す。<br>
また、ピクセルの厚さは、上下方向で3mm

In [ ]:
def display_xy_imgs(CASE, DAY, slice_list):
    # subplot用の高さ設定
    plot_line = math.ceil(len(slice_list)/5)
    
    # train画像のpath取得
    TRAIN_DIR='../input/uw-madison-gi-tract-image-segmentation/train/case'+CASE+'/case'+CASE+'_day'+DAY+'/scans/'
    train_images = glob(os.path.join(TRAIN_DIR, '**', '*.png'), recursive=True)
    train_images = sorted(train_images)
    
    # 画像表示
    fig = plt.figure(figsize=(20,plot_line*4))
    i=1
    for slice_i in slice_list:
        ax = fig.add_subplot(plot_line,5,i)
        ax.set_title('slice ' + str(slice_i))
        im = Image.open(train_images[slice_i-1])
        img = ax.imshow(np.asarray(im),cmap=plt.cm.jet)
        cbar = fig.colorbar(img, ax=ax, aspect=50, pad=0.08, shrink=0.9, orientation='vertical')
        i+=1
    plt.suptitle('case' + CASE + ' day' + DAY,fontsize=16)
    plt.show()

In [ ]:
# 見たい画像の指定
CASE = '2'
DAY = '1'
slice_list = range(1,144,10) #スライス番号 1始まり

display_xy_imgs(CASE, DAY, slice_list)

最大値で正規化などの処理はしていないので、画像ごとに値と色の対応は異なる。<br>
輝度値が255を超えている画像があるのは、少し気になるところ。

# 4. train画像上にセグメンテーションマスクを表示
参考：https://www.kaggle.com/code/awsaf49/uwmgi-mask-data

In [ ]:
def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = np.asarray(mask_rle.split(), dtype=int)
    starts = s[0::2] - 1
    lengths = s[1::2]
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape) 

def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
import tensorflow as tf
tqdm.pandas()
from matplotlib.patches import Rectangle

In [ ]:
def get_metadata(row):
    data = row['id'].split('_')
    case = int(data[0].replace('case',''))
    day = int(data[1].replace('day',''))
    slice_ = int(data[-1])
    row['case'] = case
    row['day'] = day
    row['slice'] = slice_
    return row

def path2info(row):
    path = row['image_path']
    data = path.split('/')
    slice_ = int(data[-1].split('_')[1])
    case = int(data[-3].split('_')[0].replace('case',''))
    day = int(data[-3].split('_')[1].replace('day',''))
    width = int(data[-1].split('_')[2])
    height = int(data[-1].split('_')[3])
    row['height'] = height
    row['width'] = width
    row['case'] = case
    row['day'] = day
    row['slice'] = slice_
    return row

In [ ]:
def id2mask(id_):
    idf = df_train[df_train['id']==id_]
    wh = idf[['height','width']].iloc[0]
    shape = (wh.height, wh.width, 3)
    mask = np.zeros(shape, dtype=np.uint8)
    for i, class_ in enumerate(['large_bowel', 'small_bowel', 'stomach']):
        cdf = idf[idf['class']==class_]
        rle = cdf.segmentation.squeeze()
        if len(cdf) and not pd.isna(rle):
            mask[..., i] = rle_decode(rle, shape[:2])
    return mask

def rgb2gray(mask):
    pad_mask = np.pad(mask, pad_width=[(0,0),(0,0),(1,0)])
    gray_mask = pad_mask.argmax(-1)
    return gray_mask

def gray2rgb(mask):
    rgb_mask = tf.keras.utils.to_categorical(mask, num_classes=4)
    return rgb_mask[..., 1:].astype(mask.dtype)

In [ ]:
def load_img(path):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    img = img.astype('float32') 
    img = (img - img.min())/(img.max() - img.min())*255.0 
    img = img.astype('uint8')
    return img

def show_img(img, mask=None):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img = clahe.apply(img)
    plt.figure(figsize=(10,10))
    plt.imshow(img, cmap='bone')
    
    if mask is not None:
        plt.imshow(mask, alpha=0.5)
        handles = [Rectangle((0,0),1,1, color=_c) for _c in [(0.667,0.0,0.0), (0.0,0.667,0.0), (0.0,0.0,0.667)]]
        labels = [ "Large Bowel", "Small Bowel", "Stomach"]
        plt.legend(handles,labels)
    plt.axis('off')

In [ ]:
df_train = df_train.progress_apply(get_metadata, axis=1)
df_train.head()

In [ ]:
paths = glob('../input/uw-madison-gi-tract-image-segmentation/train/*/*/*/*')
path_df = pd.DataFrame(paths, columns=['image_path'])
path_df = path_df.progress_apply(path2info, axis=1)
df_train = df_train.merge(path_df, on=['case','day','slice'])
df_train.head()

In [ ]:
def display_xy_imgs_masked(CASE, DAY, slice_list, df_train):
    # train画像のpath取得
    TRAIN_DIR='../input/uw-madison-gi-tract-image-segmentation/train/case'+CASE+'/case'+CASE+'_day'+DAY+'/scans/'
    train_images = glob(os.path.join(TRAIN_DIR, '**', '*.png'), recursive=True)
    train_images = sorted(train_images)

    # 画像表示
    i=1
    for slice_i in slice_list:
        path = train_images[slice_i-1]

        img = load_img(path)
        mask = id2mask(df_train[df_train['image_path']==path].iloc[0,0])*255

        print(df_train[df_train['image_path']==path].iloc[0,0])
        show_img(img, mask=mask)
        plt.show()
        i+=1

In [ ]:
CASE = '2'
DAY = '1'
slice_list = range(1,144,30) #スライス番号 1始まり
display_xy_imgs_masked(CASE, DAY, slice_list, df_train)

# 5. sample_submission.csv
train.csv の segmentation と同様に、予測したマスク情報をRLEエンコーディングして、スペースで区切られた形式で提出する。<br>
例えば、「1 3 10 5」は、ピクセル1、2、3、10、11、12、13、14がマスクに含まれることを意味する。

In [ ]:
sub_df=pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/sample_submission.csv')
sub_df

In [ ]:
# 提出する際は、indexをつけないように出力
sub_df.to_csv('submission.csv',index=False)